In [ ]:
import json

with open(f"free_evo_new.json") as json_file:
    data = json.load(json_file)
    circuit = data['circuit']
    print(f"CIRCUIT: {circuit}")

    qudit_1_gates = [g['gate'] for g in circuit if g['qudit'] == ['qudit_1']]
    print(f"Circuit length : {len(circuit)}")

    print(f"Qudit 1 gates: {qudit_1_gates}")
    print(f"NUM Qudit 1 gates: {len(qudit_1_gates)}")


In [ ]:
42*4

In [ ]:
def get_gates(exp_json):
    with open(f"{exp_json}") as json_file:
        data = json.load(json_file)

        circuit = data['circuit']
        print(f"Circuit length : {len(circuit)}")

        # print(f"CIRCUIT: {circuit}")

        qudit_gates = {}

        for i in range(1, 6):
            qudit_i_gates = [g['gate'] for g in circuit if g['qudit'] == [f'qudit_{i}'] ]
            qudit_gates[f'qudit_{i}'] = qudit_i_gates
            print(f"Qudit {i} gates({len(qudit_i_gates)}) : {qudit_i_gates}")




In [ ]:
def i_gate(q):
     return {
            "duration": 30,
            "qudit": [
                f"qudit_{q}"
            ],
            "gate": "IGate",
            "pulse": {}
        }
i_gate(2)

In [ ]:
import numpy as np

def rx_gate(q, theta = np.pi):
    multiplier = theta / np.pi

    return {
            "duration": 30,
            "theta": np.pi*multiplier,
            "qudit": [
                f"qudit_{q}"
            ],
            "gate": "RXGate",
            "pulse": {
                "x_drive": {
                    "sigma": 7.5,
                    "amp": 0.03113830375988602*multiplier,
                    "type": "gaussian"
                }
            }
        }

rx_gate(3, np.pi/2)

In [ ]:
import numpy as np

def ry_gate(q, theta = np.pi):
    multiplier = theta / np.pi
    return {
            "duration": 30,
            "theta": np.pi*multiplier,
            "qudit": [
                f"qudit_{q}"
            ],
            "gate": "RYGate",
            "pulse": {
                "y_drive": {
                    "sigma": 7.5,
                    "amp": 0.03113830375988602*multiplier,
                    "type": "gaussian"
                }
            }
        }
ry_gate(2, np.pi/2)

In [ ]:
import numpy as np

def rz_gate(q, theta = np.pi):
    multiplier = theta / np.pi
    return {
            "duration": 30,
            "theta": np.pi * multiplier,
            "qudit": [
                f"qudit_{q}"
            ],
            "gate": "RZGate",
            "pulse": {
                "z_drive": {
                    "sigma": 7.5,
                    "amp": 0.03113830375988602 * multiplier,
                    "type": "gaussian"
                }
            }
        }
rz_gate(2, np.pi/2)

In [ ]:
circuit = [
    i_gate(3),
    rx_gate(2),
    ry_gate(4),
    rz_gate(4),
]

circuit

In [ ]:
def Ix(theta):
    return rx_gate(2, theta)

def Sx(theta):
    return rx_gate(4, theta)

def Iy(theta):
    return ry_gate(2, theta)

def Sy(theta):
    return ry_gate(4, theta)

def Iz(theta):
    return rz_gate(2, theta)

def Sz(theta):
    return rz_gate(4, theta)

In [ ]:
from numpy import pi as pi
grover_iter = [ Iz(+pi) , Sz(+pi) , Sy(-pi/2) , Sx(+pi/2) , Sy(+pi/2) , Iy(-pi/2) , Ix(+pi/2) , Iy(+pi/2) , Iy(+pi/2) , Ix(+pi/2) , Ix(+pi/2) , Sy(+pi/2) , Sx(+pi/2) , Sx(+pi/2) , Iz(+pi) , Sz(+pi) , Sy(-pi/2) , Sx(-pi/2) , Sy(+pi/2) , Iy(-pi/2) , Ix(-pi/2) , Iy(+pi/2) , Iy(+pi/2) , Ix(+pi/2) , Ix(+pi/2) , Sy(+pi/2) , Sx(+pi/2) , Sx(+pi/2)]

In [ ]:
from numpy import pi as pi

hh = [ Iy(+pi/2) , Ix(+pi/2) , Ix(+pi/2) , Sy(+pi/2) , Sx(+pi/2) , Sx(+pi/2)]

In [ ]:
from numpy import pi as pi

def dd(q):
    return [ rx_gate(q, +pi) , ry_gate(q, +pi)]*2

In [ ]:
import json
with open(f"free_evo_new.json") as json_file:
    data = json.load(json_file)
    data['circuit'] = grover_iter

    with open('grover.json', 'w') as convert_file: 
        convert_file.write(json.dumps(data, indent=4, default=str))

In [ ]:
import json

NUM_ITER = 10
with open(f"free_evo_new.json") as json_file:
    data = json.load(json_file)
    Nq = len(data["qudits"])
    grover_ckt = hh # 6(3)

    for _ in range(NUM_ITER):
        grover_ckt.extend(grover_iter) # 28(14)

    g_count = len(grover_ckt) // 2
    i_count = 4 - (g_count % 4)

    grover_ckt.extend([i_gate(2)]*i_count)
    grover_ckt.extend([i_gate(4)]*i_count)

    dd_count = len(grover_ckt) // 8
    grover_ckt.extend(dd(1)*dd_count)
    grover_ckt.extend(dd(3)*dd_count)
    grover_ckt.extend(dd(5)*dd_count)
    # grover_ckt.extend([i_gate(1)]*dd_count*4)
    # grover_ckt.extend([i_gate(3)]*dd_count*4)
    # grover_ckt.extend([i_gate(5)]*dd_count*4)


    # print(grover_ckt)


    data['circuit'] = grover_ckt

    data['solver_opt']['saveat']['times'] = [4*30*x for x in range(dd_count+1)] 
    print(f"dd_count: {dd_count}, NEW TIMES: {data['solver_opt']['saveat']['times']} ")

    with open(f'grover_free_{NUM_ITER}.json', 'w') as convert_file: 
        convert_file.write(json.dumps(data, indent=4, default=str))

In [ ]:
get_gates('grover_free_10.json')


In [ ]:
times = [4*30*x for x in range(144//4 + 1)] 
times

In [ ]:
144*30

In [ ]:
get_gates('grover_free_10.json')


In [ ]:
get_gates('dd_spectator.json')


In [ ]:
a = []
a.extend([dd(1)]*2)
a